<a href="https://colab.research.google.com/github/rubanzasilva/abalone_age_prediction/blob/dev/ps_s04e03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [2]:
#hide
#! [ -e /content ]

#hide
#This imports and sets up everything you will need for this notebook
#
#!pip install -Uqq fastbook
#import fastbook
#fastbook.setup_book()

#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random

from fastai.imports import *
np.set_printoptions(linewidth=130)

# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path
import os
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from ipywidgets import interact


matplotlib.rc('image', cmap='Greys')

In [3]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e4:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F72489%2F8096274%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T132418Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2614deaa47c6cd905e4cdd6acd10fdf496604bbcae2b3fda248c90858a6f72c27e9c3be1ff3141903a83befe7fea039c459b9702f244616103f40546a6bab945b93e2f3f5c42517c88af809b449064671f22af0de774517811314fda01bf2c86f5a65098d31bd122c9b736d8af59944704e20719c70045136b7ee6fcd8622366df3263afeb70b6a84d8ffc2f4ad28b1362a75010e6e4b0d3d435a24fd794a0527bf8453598d8d9c988e769cec5d5c82f60ce82b0c7c784dc58788d00f248728a875a19c6755b5e08da6ca2ea38beb3003deee1dbacac986be3673f3221911468f9d844e43ab3e4d502066ff80686dd882256bff56bf83571eaa6ed2874564585'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2529839 bytes downloaded
Downloaded and uncompressed: playground-series-s4e4
Data source import complete.


In [4]:
!ls /kaggle/input/playground-series-s4e4

sample_submission.csv  test.csv  train.csv


Set random seed for reproducibility.

In [5]:
set_seed(42)

In [6]:
path = Path('/kaggle/input/playground-series-s4e4/')
path

Path('/kaggle/input/playground-series-s4e4')

Index_col

With or without.

I remove the index_col because it changes the shape of our original dataset.
 Later on we shall be able to investigate the effect of this action.

In [7]:
#train_df = pd.read_csv(path/'train.csv',index_col='id')

train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
sub_df = pd.read_csv(path/'sample_submission.csv')


In [8]:
train_df.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [9]:
sub_df.shape

(60411, 2)

In [4]:
cont_names,cat_names = cont_cat_split(train_df, max_card=20 , dep_var='Rings')

NameError: name 'cont_cat_split' is not defined

In [ ]:
cont_names,len(cont_names)

In [32]:
cat_names

['Sex']

In [33]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [36]:
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Rings',
                   splits=splits)

In [38]:
to.xs.iloc[:2]

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
id,,,,,,,,
81142,2,-0.060893,0.033288,-0.276259,-0.295537,-0.202349,-0.237821,-0.006724
55321,2,-1.627359,-1.549445,-1.457439,-1.398579,-1.391622,-1.298508,-1.517854


In [39]:
dls = to.dataloaders(bs=64)

In [40]:
dls.show_batch()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,I,0.245,0.175,0.065,0.0805,0.0235,0.0165,0.0200,5.0
1,I,0.250,0.190,0.065,0.0635,0.0255,0.0110,0.0200,5.0
2,M,0.555,0.440,0.165,0.8695,0.3325,0.1560,0.2500,13.0
3,M,0.615,0.480,0.155,0.9395,0.4275,0.1950,0.3050,9.0
4,M,0.575,0.440,0.120,0.8260,0.4400,0.1860,0.2370,9.0
5,F,0.480,0.400,0.115,0.6640,0.2725,0.1385,0.2000,12.0
6,M,0.550,0.440,0.150,0.8490,0.2825,0.1710,0.2450,13.0
7,F,0.640,0.500,0.165,1.3445,0.5320,0.2915,0.3745,9.0
8,I,0.465,0.360,0.120,0.5805,0.2370,0.1045,0.1260,8.0
9,M,0.280,0.210,0.075,0.1060,0.0400,0.0225,0.0375,7.0


In [46]:
learn = tabular_learner(dls, metrics=rmse)

In [ ]:
learn.lr_find(suggest_funcs=(slide,valley))

In [47]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,_rmse,time
0,41.783413,32.166454,5.671548,00:13
1,3.713634,3.618053,1.902118,00:13
2,3.822149,3.660600,1.913270,00:13
3,3.511412,3.658950,1.912838,00:14
4,3.870037,3.565675,1.888299,00:13
5,3.683403,3.559925,1.886776,00:14
6,3.771605,3.558787,1.886475,00:13
7,3.641761,3.548173,1.883659,00:13
8,3.562724,3.485405,1.866924,00:13
9,3.528261,3.493296,1.869036,00:13


In [ ]:
learn.show_results()

In [ ]:
#row, clas, probs = learn.predict(df.iloc[0])
#row.show()

In [ ]:
dl = learn.dls.test_dl(test_df)

In [ ]:
nn_preds = learn.get_preds(dl=dl)
nn_preds

In [ ]:
nn_preds_x = learn.get_preds()[0]
nn_preds_x

In [ ]:
to.xs[:3]

In [ ]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

Neural Network Ensemble

In [ ]:
learn.lr_find(suggest_funcs=(slide,valley))

In [ ]:
dl = learn.dls.test_dl(test_df)

In [ ]:
def ensemble():
    learn = tabular_learner(dls, metrics=RocAucMulti())
    with learn.no_bar(),learn.no_logging(): learn.fit(12, 0.014)
    return learn.get_preds(dl=dl)[0]

In [ ]:
learns = [ensemble() for _ in range(5)]

In [ ]:
ens_preds = torch.stack(learns).mean(0)

Random Forests

In [ ]:
rf = RandomForestClassifier(100, min_samples_leaf=5)
rf.fit(X_train, y_train);
#mean_absolute_error(y_test, rf.predict(X_test))

#roc_auc_score(y_test, rf.predict(X_test))

In [ ]:
rf_preds = rf.predict(X_test)

In [ ]:

#roc_auc_score(y_test, rf.predict(X_test))

XGBoost

In [ ]:
model = xgb.XGBClassifier(n_estimators = 100, max_depth=8, learning_rate=0.1, subsample=0.5)

In [ ]:
xgb_model = model.fit(X_train, y_train)

In [ ]:
xgb_preds = xgb_model.predict_proba(X_test)

In [ ]:
xgb_preds

In [ ]:
#roc_auc_score(y_test, xgb_preds)

General Ensemble

In [ ]:
avgs = (nn_preds + xgb_preds + forest_preds) / 3
avgs

In [ ]:
#roc_auc_score(y_test, avgs)

Take argmax to return preds

In [ ]:
argmax = avgs.argmax(dim=1)
argmax

In [ ]:
!kaggle competitions submit -c playground-series-s4e4 -f submission.csv -m "Message"